## Chart with activity by day of the week
This chart may be useful to identify if the behaviour changes signifiantly during the weekdays and the weekend.
For each activity, do a barchart by day, representing how many times the user have reported doing that activity in the different days of the week.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv('timediaries.csv') 
df = df.drop(["Unnamed: 0", 'sleep_quality', 'pred_day', 'daily_mood', 'problem', 'solution', 'yn_food', 'food', 'num_mood'], axis=1)
df.head()

In [ ]:
temp = pd.Timestamp('20211023')
print(temp.day_name())

In [ ]:
for row in range(len(df)):
    str_day = str(df.iloc[row].at['timestamp'])[:-4]
    day = pd.Timestamp(str_day)
    df.at[row, 'day'] = day.day_name()

In [ ]:
# may happen that a timediary is asked twice 
df.drop_duplicates(subset ="timestamp",
                     keep = False, inplace = True)
df = df.reset_index()

In [ ]:
df = df.drop(["index"], axis=1)
df

In [ ]:
d = {}

for row in range(len(df)):
    if pd.isna(df.what[row]) == False and df.what[row] != 'Expired':
        if df.day[row] not in d.keys():
            d[df.day[row]] = [df.what[row]]
        else:
            d[df.day[row]].append(df.what[row])


In [ ]:
from collections import Counter
occurrences = Counter(d['Wednesday'])
occurrences
# one count correspond to half as hour --> e.g., break I will go to sleep --> count = 28 --> 14 hours in the two Thursdays

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

keys = occurrences.keys()
values = occurrences.values()

y = np.arange(0, max(values)+1, 3)

ticks = ''
c=0
for word in keys:
    c+=1
    s = len(word)
    if '(' in word:
        s = word.find('(')
    ticks = ticks + str(c) + '. '  + word[:s] + '\n'
    
# percentage computed over the total time the activity has been reported that day of the week


# add text on right of the plot
plt.gcf().text(0.99, 0.2, ticks, fontsize=10)

# add the annotations
y_pos = np.arange(0, max(values)+1)
for y, x in zip(values, y_pos):
    plt.annotate(f'{y}\n', xy=(x, y), ha='center', va='center')

plt.bar(keys, values, align='center', alpha=0.5)
plt.xticks(np.arange(len(keys)), np.arange(1, len(keys)+1), rotation=30, horizontalalignment='center')
plt.yticks([])
plt.title('Activities')

plt.ylabel('Countings', fontsize=10)
plt.title('Activities on Thursdays', fontsize=20)
plt.show()

This is not very pretty nor telling: we can see that the activity done the most was sleeping, and this is not so relevant as one would sleep approximately 7 hours each night.
It would be propably better to re-econde activities so that they are grouped into homogeneous categories and recompute the countings.

In [ ]:
df.what.unique()

In [ ]:
# Recode activities into broader categories
for row in range(len(df)):
    if df.what[row] == 'Travelling for: Study, By bus/tram' or df.what[row]=='Travelling for: Changing locality, On foot' or \
    df.what[row] == 'Travelling for: Changing locality, By train' or df.what[row] =='Travelling for: Changing locality, By car as passenger' or \
    df.what[row] == 'Travelling for: Other leisure, By train' or df.what[row] == 'Travelling for: Study, On foot':
        df.at[row, 'what'] = 'Travelling'
    
    if df.what[row] == 'On a Break, Others' or df.what[row] == 'Other':
        df.at[row, 'what'] = 'Others'
        
    if df.what[row] == 'Walking' or df.what[row] == 'On a Break, I will participate in sports activities' or \
    df.what[row] == 'Sport: Walking, Trekking and hiking':
        df.at[row, 'what'] = 'Physical activities'
        
    if df.what[row] == 'Grocery Shopping' or df.what[row] == 'Other Shopping' or df.what[row] == 'Household and family care' \
    or df.what[row] == 'Cooking, food preparation & management':
        df.at[row, 'what'] = 'Shopping/Household activities'
        
    if df.what[row] == 'On a Break, I am starting classes/lessons/lab' or df.what[row] == 'Study/work group' \
    or df.what[row] == 'Lecture/seminar/conference/university meeting' or \
    df.what[row] == 'On a Break, I have a work/study meeting':
        df.at[row, 'what'] = 'Study/Lectures'
        
    if df.what[row] == 'Phone/Video calling; Skype/Zoom/WhatsApp/Messenger or other VoIP' or df.what[row] == 'Social media (Facebook Instagram etc.)' \
    or df.what[row] == 'Social life (Socialising, visiting, receiving, conversating with family, relatives, friends, classmate, visitors, neighbour, and others)' \
    or df.what[row] == 'Happy Hour/Drinking/Party':
        df.at[row, 'what'] = 'Social activities'
        
    if df.what[row] == 'Listening to music' or df.what[row] == 'Hobbies (assembling/repair apparatus/pc, gardening, etc.)' \
    or df.what[row] == 'Watching TV, video, YouTube, etc.' or df.what[row] == 'Reading a book, periodicals, news, etc.' or \
    df.what[row] == 'Surfed or seeking, reading information via Internet' or \
    df.what[row] == 'In chat on Internet or reading, sending e-mail':
        df.at[row, 'what'] = 'Free time/hobbies'
        
    if df.what[row] == 'Break (coffee, cigarette, drink, etc.)' \
    or df.what[row] == 'Personal care' or df.what[row] == 'Rest/nap' \
    or df.what[row] == 'Did not do anything special (Just let the time pass, Lazed around, etc.)':
        df.at[row, 'what'] = 'Breaks'
        
    if df.what[row] == 'On a Break, I will go to sleep' or df.what[row] == 'Sleeping':
        df.at[row, 'what'] = 'Sleeping'
        

Remove expired time diaries, as they give no information about the activity done, and also time diaries reporting activities such as "sleeping" and "eating", as they are not so relevant or peculiar activities, but rather activities common to every human being.

In [ ]:
d = {}

for row in range(len(df)):
    if pd.isna(df.what[row]) == False and df.what[row] != 'Expired' and df.what[row] != 'Sleeping' and df.what[row] != 'Eating':
        if df.day[row] not in d.keys():
            d[df.day[row]] = [df.what[row]]
        else:
            d[df.day[row]].append(df.what[row])

from collections import Counter
occurrences = Counter(d['Thursday'])
occurrences

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

keys = occurrences.keys()
values = occurrences.values()

color = ('red', '#00b050', '#00b0f0', 'yellow', 'lightgreen')
y_pos = np.arange(len(keys))
plt.bar(y_pos, values, align='center', color=color)

# map names to colors
cmap = dict(zip(values, color))

# create the rectangles for the legend
patches = [Patch(color=v, label=k) for k, v in cmap.items()]

# add the legend
plt.legend(title='Activities', labels=keys, handles=patches, bbox_to_anchor=(1.04, 0.5),
           loc='center left', borderaxespad=0, fontsize=15, frameon=False)

# add the annotations
for y, x in zip(values, y_pos):
    plt.annotate(f'{y}\n', xy=(x, y), ha='center', va='center')

plt.xticks(y_pos, keys, rotation=30, horizontalalignment='center')
plt.ylim(0, max(values)+5)  # this adds a little space at the top of the plot, to compensate for the annotation
plt.ylabel('Countings', fontsize=10)
plt.title('Activities', fontsize=20)


plt.show()

In [ ]:
def plot_day(day):
    from collections import Counter
    occurrences = Counter(d[day])
    
    if len(occurrences.keys()) < 8:
        for word in ['Study/Lectures', 'Breaks', 'Social activities', \
                     'Physical activities', 'Others', 'Free time/hobbies', \
                     'Shopping/Household activities', 'Travelling']:
            if word not in occurrences.keys():
                occurrences[word] = 0
                
    keys = list(occurrences.keys())
    keys.sort()
    
    values = []
    for k in keys:
        values.append(occurrences[k])
#     print(keys)
#     print(values)

    color = ('purple', 'pink', '#00b0f0', '#00b050', 'lightgreen', 'yellow', 'orange', 'red')
    y_pos = np.arange(len(keys))
    plt.bar(y_pos, values, align='center', color=color)

#     # map names to colors
#     cmap = dict(zip(values, color))

#     # create the rectangles for the legend
#     patches = [Patch(color=v, label=k) for k, v in cmap.items()]

#     # add the legend
#     plt.legend(title='Activities', labels=keys, handles=patches, bbox_to_anchor=(1.04, 0.5),
#                loc='center left', borderaxespad=0, fontsize=15, frameon=False)


    # add the annotations
    for y, x in zip(values, y_pos):
        plt.annotate(f'{y}\n', xy=(x, y), ha='center', va='center')

    plt.xticks([])
    plt.yticks([])
    plt.ylim(0, max(values)+4)  # this adds a little space at the top of the plot, to compensate for the annotation
    plt.title(day)
    
    

%matplotlib inline
import matplotlib.pyplot as plt
import math

fig = plt.figure(figsize=(20,10))

# cells:
# 1 2 3
# 4 5 6

plt.subplot(3,   # 3 rows
            3,   # 3 columns
            1)   # plotting in first cell

plot_day('Monday')


plt.subplot(3,   # 3 rows
            3,   # 3 columns
            2)   # plotting in second cell

plot_day('Tuesday')

plt.subplot(3,   # 3 rows
            3,   # 3 columns
            3)   # plotting in third cell

plot_day('Wednesday')


plt.subplot(3,   # 3 rows
            3,   # 3 columns
            4)   # plotting in fourth cell

plot_day('Thursday')


plt.subplot(3,   # 3 rows
            3,   # 3 columns
            5)   # plotting in fifth cell

plot_day('Friday')


plt.subplot(3,   # 3 rows
            3,   # 3 columns
            6)   # plotting in sixth cell

plot_day('Saturday')


plt.subplot(3,   # 3 rows
            3,   # 3 columns
            7)   # plotting in sixth cell

plot_day('Sunday')



# map names to colors
label = ['Study/Lectures', 'Breaks', 'Social activities', 'Physical activities', 'Others', 
         'Free time/hobbies', 'Shopping/Household activities', 'Travelling']
label.sort()
color = ('purple', 'pink', '#00b0f0', '#00b050', 'lightgreen', 'yellow', 'orange', 'red')
cmap = dict(zip(label, color))
# create the rectangles for the legend
patches = [Patch(color=v, label=k) for k, v in cmap.items()]
# add the legend
plt.legend(title='Activities', labels=label, handles=patches, bbox_to_anchor=(2.04, 0.6),
               loc='center left', borderaxespad=0, fontsize=15, frameon=True)


plt.subplots_adjust(wspace = 0.5, hspace = 1)  # to avoid text overlapping
plt.show()
# plt.savefig('chart-activities.png', transparent=False)

In [ ]:
occurrences = Counter(d['Monday'])
occurrences
# breaks
# free time
# others
# physical activity
# shopping
# social
# study
# travel